## MS Example 1: Multiplicative Noise, Surrogate-Based Methods

In [1]:
import numpy as np
import dfo_v1 as dfo
import ECNoise as ECN
import active_subspaces as ac
import matplotlib.pyplot as plt

We let $t: \mathbf{x}: \mathbb{R}^{10} \to \mathbb{R}$, where $t(\mathbf{x})=(x_1^2+x_2^2+x_3^2)(1+ k\epsilon)$, where $\epsilon \sim N(0,1)$ so that $k\epsilon \sim N(0,k^2)$ and $L_1=2$.

In [2]:
dim=10
k=1E-4
def tfun(x,var=k):
    y=np.copy(x)
    return 1e2*np.dot(np.transpose(y[0:3,:]),y[0:3,:])*(1+(k**.5)*np.random.randn(1))

## Sample 100 random draws in $\Lambda$, build a surrogate, minimize

In [3]:
# Perform draws
n_samps=100

X=np.random.rand(dim,n_samps)
samps=2*X-1
print(samps.shape,samps[:,0:1])

(10, 100) [[-0.37930911]
 [-0.63651892]
 [-0.22356893]
 [ 0.35803441]
 [ 0.28893941]
 [-0.82085769]
 [-0.92040034]
 [-0.02394406]
 [-0.82720953]
 [ 0.0314007 ]]


In [4]:
# Compute the f values

f_vals=np.zeros((n_samps,1))

for i in range(n_samps):
    
    f_vals[i] = tfun(samps[:,i:i+1])
    
print(f_vals.shape)
    

(100, 1)


In [5]:
ss = ac.subspaces.Subspaces()

trainx=np.transpose(samps)

# Use Global Quadratic
ss.compute(X=trainx, f=f_vals, nboot=0, sstype='QPHD')
    
RS=ac.utils.response_surfaces.PolynomialApproximation(2)
    
y1 = trainx.dot(ss.W1)
    
RS.train(y1, f_vals)

print(ss.W1)

print()

print(ss.eigenvals)

n should be an integer. Performing conversion.
[[ 9.36056118e-01  8.43293543e-02  3.41475647e-01]
 [-3.48828591e-01  3.46740817e-01  8.70659591e-01]
 [ 4.49822940e-02  9.34159265e-01 -3.53990151e-01]
 [ 4.67640647e-03  1.30449343e-03 -2.58229727e-03]
 [-2.70917407e-03  1.89643862e-03 -8.85229962e-04]
 [ 5.84936051e-03 -3.67346339e-04 -3.31775561e-03]
 [-1.98674685e-03  6.56099832e-05 -1.47282614e-03]
 [-3.89230404e-03  4.06224860e-04 -2.36955939e-03]
 [-1.88230528e-03  3.13940406e-04 -2.55650743e-03]
 [ 2.84398881e-03 -3.64161542e-04  2.11022771e-03]]

[[1.33863011e+04]
 [1.33181928e+04]
 [1.31831252e+04]
 [2.32959555e+00]
 [1.80381694e+00]
 [1.17127723e+00]
 [5.42501412e-01]
 [3.97833341e-01]
 [2.34990408e-01]
 [1.08216873e-02]]


In [6]:
avdom=ac.domains.BoundedActiveVariableDomain(ss)
    
ystar, fstar = ac.optimizers.av_minimize(lambda x: RS.predict(x.reshape(1,3))[0], avdom)

print(ystar,fstar,RS.Rsqr)

[[-0.00168002  0.00090247 -0.00287383]] 0.4115832160659415 0.9997161135203906
